In [1]:
import itertools
import numpy as np
import pulp

In [2]:
order = ['Normal', 'Fire', 'Water', 'Electric', 'Grass', 'Ice', 'Fighting', 'Poison', 'Ground', 'Flying', 'Psychic', 'Bug', 'Rock', 'Ghost', 'Dragon', 'Dark', 'Steel', 'Fairy']

In [3]:
weaknesses = {
    'Normal': [1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1],
    'Fire': [1, 0.5, 2, 1, 0.5, 0.5, 1, 1, 2, 1, 1, 0.5, 2, 1, 1, 1, 0.5, 0.5],
    'Water': [1, 0.5, 0.5, 2, 2, 0.5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0.5, 1],
    'Electric': [1, 1, 1, 0.5, 1, 1, 1, 1, 2, 0.5, 1, 1, 1, 1, 1, 1, 0.5, 1],
    'Grass': [1, 2, 0.5, 0.5, 0.5, 2, 1, 2, 0.5, 2, 1, 2, 1, 1, 1, 1, 1, 1],
    'Ice': [1, 2, 1, 1, 1, 0.5, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1],
    'Fighting': [1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 0.5, 0.5, 1, 1, 0.5, 1, 2],
    'Poison': [1, 1, 1, 1, 0.5, 1, 0.5, 0.5, 2, 1, 2, 0.5, 1, 1, 1, 1, 1, 0.5],
    'Ground': [1, 1, 2, 0, 2, 2, 1, 0.5, 1, 1, 1, 1, 0.5, 1, 1, 1, 1, 1],
    'Flying': [1, 1, 1, 2, 0.5, 2, 0.5, 1, 0, 1, 1, 0.5, 2, 1, 1, 1, 1, 1],
    'Psychic': [1, 1, 1, 1, 1, 1, 0.5, 1, 1, 1, 0.5, 2, 1, 2, 1, 2, 1, 1],
    'Bug': [1, 2, 1, 1, 0.5, 1, 0.5, 1, 0.5, 2, 1, 1, 2, 1, 1, 1, 1, 1],
    'Rock': [0.5, 0.5, 2, 1, 2, 1, 2, 0.5, 2, 0.5, 1, 1, 1, 1, 1, 1, 2, 1],
    'Ghost': [0, 1, 1, 1, 1, 1, 0, 0.5, 1, 1, 1, 0.5, 1, 2, 1, 2, 1, 1],
    'Dragon': [1, 0.5, 0.5, 0.5, 0.5, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2],
    'Dark': [1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 0, 2, 1, 0.5, 1, 0.5, 1, 2],
    'Steel': [0.5, 2, 1, 1, 0.5, 0.5, 2, 0, 2, 0.5, 0.5, 0.5, 0.5, 1, 0.5, 1, 0.5, 0.5],
    'Fairy': [1, 1, 1, 1, 1, 1, 0.5, 2, 1, 1, 1, 0.5, 1, 1, 0, 0.5, 2, 1]
}

In [4]:
# https://pokemondb.net/type/unused
unused = {
    ('Normal', 'Ice'),
    ('Normal', 'Rock'),
    ('Fire', 'Grass'),
    ('Electric', 'Fighting'),
    ('Ice', 'Poison'),
    ('Fighting', 'Fairy'),
    ('Poison', 'Fairy'),
    ('Bug', 'Dragon'),
    ('Dark', 'Fairy'),
    ('Normal', 'Poison'),
    ('Normal', 'Ghost'),
    ('Fire', 'Ice'),
    ('Electric', 'Poison'),
    ('Ice', 'Bug'),
    ('Poison', 'Psychic'),
    ('Ground', 'Fairy'),
    ('Bug', 'Dark'),
    ('Normal', 'Bug'),
    ('Normal', 'Steel'),
    ('Fire', 'Fairy'),
    ('Electric', 'Dark'),
    ('Fighting', 'Ground'),
    ('Poison', 'Steel'),
    ('Psychic', 'Bug'),
    ('Rock', 'Ghost'),
    'Flying',
    ('Electric', 'Dragon'),
    ('Fighting', 'Ghost'),
    ('Ghost', 'Dragon'),
    ('Fire', 'Water'),
    ('Ice', 'Dragon'),
    ('Psychic', 'Ghost'),
    ('Dragon', 'Steel'),
    ('Fire', 'Steel'),
    ('Fighting', 'Rock'),
    ('Psychic', 'Dragon'),
    ('Electric', 'Dragon'),
    ('Psychic', 'Dragon'),
    ('Dragon', 'Fairy')
}

In [5]:
pair_order = order[:]
for pair in itertools.combinations(order, 2):
    if pair not in unused:
        first = pair[0]
        second = pair[1]
        first_vector = np.array(weaknesses[first])
        second_vector = np.array(weaknesses[second])
        pair_vector = first_vector * second_vector
        weaknesses[pair] = list(pair_vector)
        pair_order.append(pair)

In [6]:
immune = [min([weaknesses[pair][i] for pair in weaknesses.keys()]) == 0 for i in range(18)]
can_immune = [[1 if weaknesses[pair][t] == 0 else 0 for t in range(len(order))] for pair in pair_order]
can_resist = [[1 if (weaknesses[pair][t] == 0.5 or weaknesses[pair][t] == 0.25) else 0 for t in range(len(order))] for pair in pair_order]

# Objectives
weak = [sum([1 if w > 1 else 0 for w in weaknesses[pair]]) for pair in pair_order]
superweak = [sum([1 if w == 4 else 0 for w in weaknesses[pair]]) for pair in pair_order]
resist = [sum([1 if w < 1 else 0 for w in weaknesses[pair]]) for pair in pair_order]
superresist = [sum([w if w == 0.25 else 0 for w in weaknesses[pair]]) for pair in pair_order]
total = [sum(weaknesses[pair]) for pair in pair_order]

# 1. Min weak

In [7]:
prob = pulp.LpProblem("PokemonTypeCombos", pulp.LpMinimize)
x = pulp.LpVariable.dicts("x", range(len(pair_order)), cat=pulp.LpBinary)
objective_function = sum([weak[t] * x[t] for t in range(len(pair_order))])
prob += objective_function
prob += sum([x[t] for t in range(len(pair_order))]) == 6
for tp in range(len(order)):
    if immune[tp]:
        prob += sum([can_immune[pair][tp] * x[pair] for pair in range(len(pair_order))]) >= 1
        prob += sum([can_resist[pair][tp] * x[pair] for pair in range(len(pair_order))]) >= 1
    else:
        prob += sum([can_resist[pair][tp] * x[pair] for pair in range(len(pair_order))]) >= 2
prob.solve()

1

In [8]:
for i in range(len(pair_order)):
    if x[i].value() > 0:
        print(pair_order[i])

Normal
('Fire', 'Ground')
('Water', 'Flying')
('Grass', 'Ghost')
('Dark', 'Steel')
('Steel', 'Fairy')


# 2. Min superweak

In [9]:
prob = pulp.LpProblem("PokemonTypeCombos", pulp.LpMinimize)
x = pulp.LpVariable.dicts("x", range(len(pair_order)), cat=pulp.LpBinary)
objective_function = sum([superweak[t] * x[t] for t in range(len(pair_order))])
prob += objective_function
prob += sum([x[t] for t in range(len(pair_order))]) == 6
for tp in range(len(order)):
    if immune[tp]:
        prob += sum([can_immune[pair][tp] * x[pair] for pair in range(len(pair_order))]) >= 1
        prob += sum([can_resist[pair][tp] * x[pair] for pair in range(len(pair_order))]) >= 1
    else:
        prob += sum([can_resist[pair][tp] * x[pair] for pair in range(len(pair_order))]) >= 2
prob.solve()

1

In [10]:
for i in range(len(pair_order)):
    if x[i].value() > 0:
        print(pair_order[i])

('Normal', 'Grass')
('Fire', 'Ghost')
('Water', 'Fairy')
('Ground', 'Dark')
('Flying', 'Steel')
('Psychic', 'Steel')


# 3. Max resist

In [11]:
prob = pulp.LpProblem("PokemonTypeCombos", pulp.LpMaximize)
x = pulp.LpVariable.dicts("x", range(len(pair_order)), cat=pulp.LpBinary)
objective_function = sum([resist[t] * x[t] for t in range(len(pair_order))])
prob += objective_function
prob += sum([x[t] for t in range(len(pair_order))]) == 6
for tp in range(len(order)):
    if immune[tp]:
        prob += sum([can_immune[pair][tp] * x[pair] for pair in range(len(pair_order))]) >= 1
        prob += sum([can_resist[pair][tp] * x[pair] for pair in range(len(pair_order))]) >= 1
    else:
        prob += sum([can_resist[pair][tp] * x[pair] for pair in range(len(pair_order))]) >= 2
prob.solve()

1

In [12]:
for i in range(len(pair_order)):
    if x[i].value() > 0:
        print(pair_order[i])

('Normal', 'Fire')
('Water', 'Ghost')
('Grass', 'Fairy')
('Ground', 'Steel')
('Flying', 'Steel')
('Dark', 'Steel')


# 4. Max superresist

In [13]:
prob = pulp.LpProblem("PokemonTypeCombos", pulp.LpMaximize)
x = pulp.LpVariable.dicts("x", range(len(pair_order)), cat=pulp.LpBinary)
objective_function = sum([superresist[t] * x[t] for t in range(len(pair_order))])
prob += objective_function
prob += sum([x[t] for t in range(len(pair_order))]) == 6
for tp in range(len(order)):
    if immune[tp]:
        prob += sum([can_immune[pair][tp] * x[pair] for pair in range(len(pair_order))]) >= 1
        prob += sum([can_resist[pair][tp] * x[pair] for pair in range(len(pair_order))]) >= 1
    else:
        prob += sum([can_resist[pair][tp] * x[pair] for pair in range(len(pair_order))]) >= 2
prob.solve()

1

In [14]:
for i in range(len(pair_order)):
    if x[i].value() > 0:
        print(pair_order[i])

('Normal', 'Ground')
('Fire', 'Ghost')
('Water', 'Steel')
('Grass', 'Dragon')
('Flying', 'Fairy')
('Rock', 'Dark')


# 5. Min total
### (At least 1 immunity, at least 1 resistance)

In [15]:
prob = pulp.LpProblem("PokemonTypeCombos", pulp.LpMinimize)
x = pulp.LpVariable.dicts("x", range(len(pair_order)), cat=pulp.LpBinary)
objective_function = sum([total[t] * x[t] for t in range(len(pair_order))])
prob += objective_function
prob += sum([x[t] for t in range(len(pair_order))]) == 6
for tp in range(len(order)):
    if immune[tp]:
        prob += sum([can_immune[pair][tp] * x[pair] for pair in range(len(pair_order))]) >= 1
        prob += sum([can_resist[pair][tp] * x[pair] for pair in range(len(pair_order))]) >= 1
    else:
        prob += sum([can_resist[pair][tp] * x[pair] for pair in range(len(pair_order))]) >= 2
prob.solve()

1

In [16]:
for i in range(len(pair_order)):
    if x[i].value() > 0:
        print(pair_order[i])

('Normal', 'Dragon')
('Water', 'Dark')
('Ground', 'Bug')
('Flying', 'Steel')
('Ghost', 'Steel')
('Steel', 'Fairy')
